In [5]:
TOTAL_CHAR = 2**16


# derivative first half (log P(s) / alpha)
import numpy as np

import sys
import pickle
import math
import numpy as np
import pdb

import italian_vocab
import chinese

# n = 4   # number of letters as context
MAX_N = 4


class Blending_with_linear_param:
    max_f = 0
    
    def __init__(self, max_f):
        self.max_f = max_f
        return None
    
    def derivative_of_P_to_alpha(self, alpha_matrix, beta_matrix, model, prefix, x):
        
        n = len(prefix)
        alpha, beta = self.compute_alpha_beta(alpha_matrix, beta_matrix, prefix, model)
        if prefix in model:
            ms = sum(model[prefix].values())
            us = len(model[prefix])
            if x in model[prefix]:
                ms_x = model[prefix][x]
            else:
                ms_x = 0
        else:
            ms = 0
            us = 0
            ms_x = 0
            
        eq1 = (beta-ms_x) / (ms+alpha)**2
        
        if len(prefix) == 0:
            eq2 = (ms-us*beta) / (ms+alpha)**2 / TOTAL_CHAR
            eq3 = 0
        else:
            next_prefix = prefix[1:]
            eq2 = (ms-us*beta) / (ms+alpha)**2 * self.blend(alpha_matrix, beta_matrix, next_prefix, x, model)
            eq3 = (alpha + us*beta) / (ms+alpha) * self.derivative_of_P_to_beta(alpha_matrix, beta_matrix, model, next_prefix, x)
        
        return eq1 + eq2 + eq3
            
    def derivative_of_P_to_beta(self, alpha_matrix, beta_matrix, model, prefix, x):
        
        n = len(prefix)
        alpha, beta = self.compute_alpha_beta(alpha_matrix, beta_matrix, prefix, model)
        if prefix in model:
            ms = sum(model[prefix].values())
            us = len(model[prefix])
            if x in model[prefix]:
                ms_x = model[prefix][x]
            else:
                ms_x = 0
        else:
            ms = 0
            us = 0
            ms_x = 0
            
        eq1 = -1 / (ms+alpha)
        if len(prefix) == 0:
            eq2 = (ms-us*beta) / (ms+alpha)**2 / TOTAL_CHAR
            eq3 = 0
        else:
            next_prefix = prefix[1:]
            eq2 = us / (ms+alpha) * self.blend(alpha_matrix, beta_matrix, next_prefix, x, model, n)
            eq3 = (alpha + us*beta) / (ms+alpha) * self.derivative_of_P_to_alpha(alpha_matrix, beta_matrix, model, next_prefix, x)
        
        return eq1 + eq2 + eq3
            

    def update_param(self, alpha_matrix, beta_matrix, prefix, x, model):
        learning_rate = 0.003
        n = len(prefix)
        # add a computation for alpha beta
        param_derivative_mapping = [1, len(set(prefix))]
        
        
        for ind, alpha_derivative in zip(range(len(alpha_matrix[n])), param_derivative_mapping):
            derivative_alpha = 1 / self.blend(alpha_matrix, beta_matrix, prefix, x, model, n) * self.derivative_of_P_to_alpha(alpha_matrix, beta_matrix, model, prefix, x) * alpha_derivative
            alpha_matrix[n][ind] += learning_rate * derivative_alpha
            
        for ind, beta_derivative in zip(range(len(beta_matrix[n])), param_derivative_mapping):
            derivative_beta = 1 / self.blend(alpha_matrix, beta_matrix, prefix, x, model, n) * self.derivative_of_P_to_beta(alpha_matrix, beta_matrix, model, prefix, x) * beta_derivative
            beta_matrix[n][ind] += learning_rate * derivative_beta
            
        # restrict param for beta
        beta_matrix[n][0] = max(0, min(1, beta_matrix[n][0]))
        
        if beta_matrix[n][0] + self.max_f * beta_matrix[n][1] > 1:
            beta_matrix[n][1] = (1 - beta_matrix[n][0]) / self.max_f
        elif beta_matrix[n][0] + self.max_f * beta_matrix[n][1] < 0:
            beta_matrix[n][1] = -beta_matrix[n][0] / self.max_f
            
        max_beta = max(beta_matrix[n][0], self.max_f*beta_matrix[n][1])
        # restrict param for alpha
        alpha_matrix[n][0] = max(alpha_matrix[n][0], -max_beta)
        
        if alpha_matrix[n][0] + self.max_f * alpha_matrix[n][1] < 0:
            alpha_matrix[n][1] = -alpha_matrix[n][0] / self.max_f
        
            

    def compute_alpha_beta(self, alpha_matrix, beta_matrix, prefix, model):
        n = len(prefix)
        if prefix in model:
            f = len(model[prefix])
        else:
            f = 0
        
        if f > self.max_f:
            f = self.max_f
            
        param_derivative_mapping = [1, f]
        alpha = np.dot(alpha_matrix[n], param_derivative_mapping)
        beta = np.dot(beta_matrix[n], param_derivative_mapping)
        
        #beta = max(0, min(1, beta))
        # alpha = max(-beta, alpha)
        
        
        return (alpha, beta)

    def build_model_with_train(self, data, alpha_matrix, beta_matrix, word_file, n):
        """
        Build n-gram model of the words in  words_lang.txt
        """
        n_grams = {}
        # read in all n+1 grams
        n_plus_1_gram_counts = {}
        for word in data:
            # update n-gram
            if not word or word[0] == "#" or " " in word:
                continue
            word = "^" * n + word.strip() + "$"
            for i in range(len(word) - n):
                n_plus_1_gram = word[i:i + n + 1]
                    
                for N in range(n, -1, -1):
                    x = n_plus_1_gram[-1]
                    next_n = n_plus_1_gram[:-1]
                    
                    if next_n not in n_grams:
                        n_grams[next_n] = {}
                        
                    if x not in n_grams[next_n]:
                        n_grams[next_n][x] = 1
                    else:
                        n_grams[next_n][x] += 1
                        
                    n_plus_1_gram = n_plus_1_gram[1:]
                    
            # update alpha and beta
            for i in range(len(word) - n):
                n_plus_1_gram = word[i:i + n + 1]
                    
                for N in range(n, -1, -1):
                    x = n_plus_1_gram[-1]
                    next_n = n_plus_1_gram[:-1]
                    
                    self.update_param(alpha_matrix, beta_matrix, next_n, x, n_grams)
                        
                    n_plus_1_gram = n_plus_1_gram[1:]

        return n_grams
    
    def blend(self, alpha_matrix, beta_matrix, prefix, x, model, n=4):
        prob = 0
        
        # compute alpha and beta
        alpha, beta = self.compute_alpha_beta(alpha_matrix, beta_matrix, prefix, model)
        possible_x = TOTAL_CHAR

        # first half of algorithm
        if prefix in model:
            base = (len(model[prefix])*beta + alpha) / (sum(model[prefix].values()) + alpha)
            if x in model[prefix]:
                prob += (model[prefix][x] - beta) / (sum(model[prefix].values()) + alpha)
        else:
            base = (0 + alpha) / (0 + alpha)
            
        if len(prefix) == 0:
            prob += base / possible_x
        else:
            prob += base * self.blend(alpha_matrix, beta_matrix, prefix[1:], x, model, n)
            
        return prob


    def word_prob_blend(self, alpha_matrix, beta_matrix, word, model, n=4):
        word = "^" * n + word.strip() + "$"
        pos = n  # char after n ^
        log_likelihood = 0
        # print ("   ", end="")
        while pos < len(word):
            prefix = word[pos - n:pos]
            prob = self.blend(alpha_matrix, beta_matrix, prefix, word[pos], model, n)
            # print(prefix, word[pos], prob)
            if prob:
                log_likelihood += math.log (prob)
            pos += 1


        return log_likelihood / (len(word) - 1)  # didn't guess "^"



In [6]:
# trainer with 2d matrix for d and f

class Blending_with_2d_matrix(Blending_with_linear_param):
    def init(self, max_f):
        super(max_f)
        return None
    
    def update_param(self, alpha_matrix, beta_matrix, prefix, x, model):
        learning_rate = 0.003
        d = len(prefix)
        n = d
        if prefix in model:
            f = len(model[prefix])
        else:
            f = 0
        
        if f > self.max_f:
            f = self.max_f
            
        derivative_beta = 1 / self.blend(alpha_matrix, beta_matrix, prefix, x, model, n) * self.derivative_of_P_to_beta(alpha_matrix, beta_matrix, model, prefix, x)
        beta_matrix[d][f] += learning_rate * derivative_beta
        
        beta_matrix[d][f] = max(0, min(1, beta_matrix[d][f]))
        
        derivative_alpha = 1 / self.blend(alpha_matrix, beta_matrix, prefix, x, model, n) * self.derivative_of_P_to_alpha(alpha_matrix, beta_matrix, model, prefix, x)
        alpha_matrix[d][f] += learning_rate * derivative_alpha
        
        alpha_matrix[d][f] = max(-beta_matrix[d][f], alpha_matrix[d][f])
        
        
    
    def compute_alpha_beta(self, alpha_matrix, beta_matrix, prefix, model):
        d = len(prefix)
        
        if prefix in model:
            f = len(model[prefix])
        else:
            f = 0
        
        if f > self.max_f:
            f = self.max_f
        
        return (alpha_matrix[d][f], beta_matrix[d][f])

In [58]:
# test build model and train
MAX_F = 10
n = 5

# load data
data = []
with open ("words_manual_en.txt", "r") as f :
    for word in f:
        data.append(word)

alpha_matrix = [[0.5, 0] for _ in range(n+1)]
beta_matrix = [[0.75, 0] for _ in range(n+1)]

trainer_1 = Blending_with_linear_param(MAX_F)
n_gram_model_1 = trainer_1.build_model_with_train(alpha_matrix, beta_matrix, "words_manual_en.txt", n)

# verify
cand = [chr(i) for i in range(ord('A'), ord('Z') + 1)] + [chr(i) for i in range(ord('a'), ord('z') + 1)] + ["$"]

# alpha_matrix = [[1, 0], [1, 0], [1, 0], [1, 0], [1, 0]]
# beta_matrix = [[1, 0], [1, 0], [1, 0], [1, 0], [1, 0]]
def verify(trainer, prefix, model):
    s = 0
    for c in cand:
        prob = trainer.blend(alpha_matrix, beta_matrix, prefix, c, model, n)
        s += prob
        
    return s

print(verify(trainer_1, "ralv", n_gram_model_1))
print(verify(trainer_1, "^All", n_gram_model_1))
print(verify(trainer_1, "Trum", n_gram_model_1))
print(verify(trainer_1, "Russ", n_gram_model_1))
print(verify(trainer_1, "Koch", n_gram_model_1))
print(verify(trainer_1, "Rack", n_gram_model_1))

0.9999999999999999
1.0
0.9999999999999999
1.0
0.9999999999999999
1.0


In [4]:
# test build model and train
MAX_F = 10
n = 5

# load data
data = []
with open ("words_manual_en.txt", "r") as f :
    for word in f:
        data.append(word)

alpha_matrix = [[0.5 for _ in range(MAX_F+1)] for _ in range(n+1)]
beta_matrix = [[0.75 for _ in range(MAX_F+1)] for _ in range(n+1)]

trainer_2 = Blending_with_2d_matrix(MAX_F)
n_gram_model_2 = trainer_2.build_model_with_train(data, alpha_matrix, beta_matrix, "words_manual_en.txt", n)

# verify
cand = [chr(i) for i in range(ord('A'), ord('Z') + 1)] + [chr(i) for i in range(ord('a'), ord('z') + 1)] + ["$"]

def verify(trainer, prefix, model, n=n):
    s = 0
    for c in cand:
        prob = trainer.blend(alpha_matrix, beta_matrix, prefix, c, model)
        
        s += prob
        
    return s

print(verify(trainer_2, "ralv", n_gram_model_2))
print(verify(trainer_2, "^All", n_gram_model_2))
print(verify(trainer_2, "Trum", n_gram_model_2))
print(verify(trainer_2, "Russ", n_gram_model_2))
print(verify(trainer_2, "Koch", n_gram_model_2))
print(verify(trainer_2, "Rack", n_gram_model_2))

1.0
0.999999999744496
1.0
1.0
1.0
1.0


In [ ]:
beta_matrix

[[0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.6850448508074591, 0.75, 0.75, 0.75, 0],
 [0.75,
  0.38409422131037535,
  0.4364834595076074,
  0.44368669244751446,
  0.49456958006947976,
  0.4758418591768404,
  0.5432620677200376,
  0.5889374387396005,
  0.5721489144783766,
  0.6248700104831192,
  0.00022023063195687254],
 [0.75,
  0,
  0,
  0,
  0.07090271169346975,
  0.2580670172289869,
  0.3898318945902937,
  0.4647159029352439,
  0.5597668408931874,
  0.567961018414197,
  0.3283024740495148],
 [0.75,
  0,
  0.0018261506595701968,
  0.18288404908030825,
  0.38100256242195457,
  0.5123989433527012,
  0.6013934670050458,
  0.6690310715422526,
  0.6850879957947016,
  0.7127280704797957,
  0.5436575850432567],
 [0.75,
  0,
  0.2504879866991498,
  0.4465826846203506,
  0.510607161145451,
  0.5728522828300227,
  0.6229852543617678,
  0.677999897204668,
  0.6848499927185444,
  0.7154580369806254,
  0.5376696016640183],
 [0.75,
  0,
  0.39500743908400676,
  0.516119825427136,
  0.5309930287467219,


Compare training and non training performance

In [8]:
# test build model and train for linear equation pram method
MAX_F = 10
n = 5

# load data
data = []
with open ("words_manual_en.txt", "r") as f :
    for word in f:
        data.append(word)

alpha_matrix = [[0.5, 0] for _ in range(n+1)]
beta_matrix = [[0.75, 0] for _ in range(n+1)]

trainer_1 = Blending_with_linear_param(MAX_F)
n_gram_model_1 = trainer_1.build_model_with_train(data, alpha_matrix, beta_matrix, "words_manual_en.txt", n)

alpha_matrix_untrained = [[0.5, 0] for _ in range(n+1)]
beta_matrix_untrained = [[0.75, 0] for _ in range(n+1)]

iter = 1000
count = 0
with open ("words_manual_en.txt", "r") as f :
    total_un = 0
    total = 0
    for word in f:
        word = word.strip()
        untrained_score = trainer_1.word_prob_blend (alpha_matrix_untrained, beta_matrix_untrained, word, n_gram_model_1, n)
        trained_score = trainer_1.word_prob_blend (alpha_matrix, beta_matrix, word, n_gram_model_1, n)
        
        total_un += untrained_score
        total += trained_score
        
        count += 1
        if count == iter:
            break
        
    print(f"avg untrained: {-total_un/iter}, avg trained: {-total/iter}")

avg untrained: 0.41165258236321056, avg trained: 0.3019432506125171


In [14]:
# test build model and train for matrix method
import random

data_amount = 10000
count = 0

# load data
data = []
with open ("../checked_words.txt", "r") as f :
    for word in f:
        if not word or word[0] == "#" or " " in word:
            continue
        
        data.append(word)
        
random.shuffle(data)
data = data[:data_amount]

MAX_F = 10
n = 5
alpha_matrix = [[0.5 for _ in range(MAX_F+1)] for _ in range(n+1)]
beta_matrix = [[0.75 for _ in range(MAX_F+1)] for _ in range(n+1)]

trainer_2 = Blending_with_2d_matrix(MAX_F)
n_gram_model_2 = trainer_2.build_model_with_train(data, alpha_matrix, beta_matrix, "words_manual_en.txt", n)

alpha_matrix_untrained = [[0.5 for _ in range(MAX_F+1)] for _ in range(n+1)]
beta_matrix_untrained = [[0.75 for _ in range(MAX_F+1)] for _ in range(n+1)]
    
    
total_un = 0
total = 0
for word in data:
    word = word.strip()
    untrained_score = trainer_2.word_prob_blend (alpha_matrix_untrained, beta_matrix_untrained, word, n_gram_model_2, n)
    trained_score = trainer_2.word_prob_blend (alpha_matrix, beta_matrix, word, n_gram_model_2, n)
    
    total_un += untrained_score
    total += trained_score
    
    count += 1
    if count == data_amount:
        break
    
print(f"avg untrained: {-total_un/data_amount}, avg trained: {-total/data_amount}")

avg untrained: 1.0146175802848714, avg trained: 0.7400299799082519


Compare the effect of input sequence

In [12]:
# test input sequence for matrix method
import random
MAX_F = 10
n = 5


for i in range(1):
    # load data
    data = []
    with open ("../checked_words.txt", "r") as f :
        for word in f:
            if not word or word[0] == "#" or " " in word:
                continue
            
            data.append(word)
            
    random.shuffle(data)
    data = data[:10000]

    alpha_matrix = [[0.5, 0] for _ in range(n+1)]
    beta_matrix = [[0.75, 0] for _ in range(n+1)]

    trainer_linear = Blending_with_linear_param(MAX_F)
    n_gram_model_linear = trainer_linear.build_model_with_train(data, alpha_matrix, beta_matrix, "words_manual_en.txt", n)
    
    iter = 100000
    count = 0
    
    total_un = 0
    total = 0
    for word in data:
        word = word.strip()
        trained_score = trainer_linear.word_prob_blend (alpha_matrix, beta_matrix, word, n_gram_model_linear, n)
        
        total += trained_score
        
        count += 1
        if count == iter:
            break
        
    print(f"iter: {i}, avg trained: {-total/iter}")

iter: 0, avg trained: 0.07399423562446077


In [13]:
# test input sequence for matrix method
import random
MAX_F = 10
n = 5


for i in range(1):
    # load data
    data = []
    with open ("../checked_words.txt", "r") as f :
        for word in f:
            data.append(word)
            
    random.shuffle(data)
    
    data = data[:10000]

    alpha_matrix = [[0.5 for _ in range(MAX_F+1)] for _ in range(n+1)]
    beta_matrix = [[0.75 for _ in range(MAX_F+1)] for _ in range(n+1)]

    trainer_matrix = Blending_with_2d_matrix(MAX_F)
    n_gram_model_matrix = trainer_matrix.build_model_with_train(data, alpha_matrix, beta_matrix, "words_manual_en.txt", n)
    
    iter = 10000
    count = 0
    
    total_un = 0
    total = 0
    for word in data:
        word = word.strip()
        trained_score = trainer_matrix.word_prob_blend (alpha_matrix, beta_matrix, word, n_gram_model_matrix, n)
        
        total += trained_score
        
        count += 1
        if count == iter:
            break
        
    print(f"iter: {i}, avg trained: {-total/iter}")

iter: 0, avg trained: 0.7501159899187466


In [ ]:
# unused code
def build_model(self, word_file, n):
        """
        Build n-gram model of the words in  words_lang.txt
        """
        # read in all n+1 grams
        n_plus_1_gram_counts = {}
        with open(word_file, "r") as f:
            for word in f:
                if not word or word[0] == "#" or " " in word:
                    continue
                word = "^" * n + word.strip() + "$"
                for i in range(len(word) - n):
                    n_plus_1_gram = word[i:i + n + 1]
                    if n_plus_1_gram not in n_plus_1_gram_counts:
                        n_plus_1_gram_counts[n_plus_1_gram] = 1
                    else:
                        n_plus_1_gram_counts[n_plus_1_gram] += 1
        # with open ("tmpppp", "w") as f :
        #  for key in sorted(n_plus_1_gram_counts) :
        #    print (str({key: n_plus_1_gram_counts[key]})[1:-1], file=f)

        # Find conditional probability of n+1st from previous n-gram
        counts = {}
        n_grams = {x: {} for x in range(0, n + 1)}
        for N in range(n, -1, -1):
            n_gram = None
            for ng in sorted(n_plus_1_gram_counts):
                next_n = ng[:-1]
                if n_gram != next_n:
                    if n_gram == None:
                        n_gram = next_n
                    else:
                        s = sum([counts[x] for x in counts])
                        if not n_gram.startswith('^^'):  # at most one ^ at the start
                            n_grams[N][n_gram] = {c: counts[c] for c in counts}

                        n_gram = next_n
                        counts = {}

                counts[ng[-1]] = n_plus_1_gram_counts[ng]

            # process last case.  Should we put a sentinel in n_plus_1_gram_counts?
            s = sum([counts[x] for x in counts])
            n_grams[N][n_gram] = {c: counts[c] for c in counts}

            if N > 0:
                new_n = {}
                for key in n_plus_1_gram_counts:
                    suff = key[1:]
                    if suff in new_n:
                        new_n[suff] += 1
                    else:
                        new_n[suff] = 1
                n_plus_1_gram_counts = new_n

        # compress to variable n?
        # print(n_grams)

        for N in range(n):
            n_grams[N + 1].update(n_grams[N])

        return n_grams[n]
    
def word_prob(self, word, model):
    word = "^" + word + "$"
    pos = 1  # char after ^
    log_likelihood = 0
    # print ("   ", end="")
    while pos < len(word):
        done = False
        for i in range(pos):
            history = word[i:pos]
            if history in model:
                if history not in model:
                    # Should penalize this
                    history = history.lower()
                try:
                    log_likelihood += math.log(model[history][word[pos]] / sum(model[history].values()))
                except:
                    low = word[pos].lower()
                    if low in model[history]:
                        # Should penalize this
                        log_likelihood += math.log(model[history][low] / sum(model[history].values()))
                    else:
                        # pdb.set_trace ()
                        log_likelihood += -20
                # print (int(log_likelihood), end = " ")
                log_likelihood -= 3 * i  # penalize shorter histories
                done = True
                pos += 1
                break

        if not done:  # Transisition so unlikely, it was never seen
            # Should use "smoothing", but this will disappear
            # when the variable-length prefixes are implemented
            log_likelihood += -20
            # print (int(log_likelihood), end = " ")
            pos += 1

    # print()

    return log_likelihood / (len(word) - 1)  # didn't guess "^"
